In [48]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from credentials import *
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
import random
import numpy

#convert string to float
import locale
locale.setlocale(locale.LC_ALL, '')

from requests import request
import json
from pandas.io.json import json_normalize

#Start browser
#chrome_options = Options()
#browser = webdriver.Chrome(ChromeDriverManager().install())
#browser.maximize_window()


In [59]:
# Key randomizer
# Arbitratily set standard deviation. I chose what ever made the automated typing look more natural
std_dev = 0.02

# Used to type a list of words to the specified element from a Selenium object
def delayedTyping(wpm, words, element, extrakey=None):

    num_of_chars = len(words)


    # This calculates the average time between keystrokes needed to roughly obtain the desired WPM
    mean_keystroke_time = 30/(num_of_chars*wpm)

    # Absolute value is taken since sleep function only allows values >=0
    keystroke_time = abs(numpy.random.normal(mean_keystroke_time, std_dev, num_of_chars))

    for i in range(num_of_chars):
        element.send_keys(words[i])
        time.sleep(keystroke_time[i])
        
    if extrakey == 1:
        element.send_keys(Keys.ENTER)
        print('enter')
    elif extrakey == 2:
        element.send_keys(Keys.DOWN)
        element.send_keys(Keys.ENTER)
    elif extrakey ==3:
        sleep(2)
        element.send_keys(Keys.ENTER)
        sleep(2)
        element.send_keys(Keys.ENTER)
    else:
        return
def random_action():
    return
        
def login():
    def cookies():
        try:
            findcookies = browser.find_element(By.XPATH, "//button[contains(text(), 'necessary cookies')]")
            findcookies.click()
        except NoSuchElementException:
            return
    sign_in_url = 'https://prosperousuniverse.com/auth/login/'
    browser.get(sign_in_url)
    sleep(3)
    cookies()
    email = browser.find_element(By.XPATH, "//input[@name='login']")
    password = browser.find_element(By.XPATH, "//input[@name='password']")
    email.send_keys(EMAIL)
    password.send_keys(PASSWORD)
    loginbtn = browser.find_element(By.XPATH, "//button[contains(text(), 'Log in')]")
    sleep(2)
    loginbtn.click()
    sleep(2)
    main_menu()
    # Add login complete check
    
    
    
def main_menu():
    apex_url = 'https://apex.prosperousuniverse.com/#/'
    browser.get(apex_url)
    sleep(4)
    try:
        element = browser.find_element_by_xpath("//div[@class='_4u9Y9yeM9aBVyYveknLIiQ==']")
        browser.execute_script("""var element = arguments[0]; 
        element.parentNode.removeChild(element);""", element)
    except Exception:
        print(Exception)
        pass
    
def sleep(t):
    randnum = random.randint(2, t)
    print('random sleep: ' + str(randnum) + 's')
    time.sleep(randnum)
    
def buffer(buffer_command, extrakey=None):    
    try:
        print('close buffer')
        close_buffer()
    except:
        pass
    print('new buffer')
    new_bfr = browser.find_element(By.XPATH, "//div[contains(text(), 'NEW BFR')]")
    new_bfr.click()
    sleep(7)
    bfr_cmd_loc = browser.find_element(By.XPATH, "//input[@placeholder='Enter content command']")
    delayedTyping(90, buffer_command, bfr_cmd_loc, extrakey)
    sleep(2)
    print('new buffer created')
def close_buffer(buffer_tab=1):
        bfr_loc = browser.find_element(By.XPATH, f"(//div[@title='close'])[{buffer_tab}]")
        bfr_loc.click()
    
def ship_status():
    buffer('FLT', 3)
    dfs = get_table()
    df = dfs[0]
    return df
    

def ship_fly(ship, LOC):
    def ship_fly_confirm():
        sleep(2)
        destination_loc = browser.find_element(By.XPATH, "//button[contains(text(), 'start')]")
        destination_loc.click()
        sleep(7)
        destination_loc = browser.find_element(By.XPATH, "//button[2][contains(text(), 'start')]")
        destination_loc.click()
        
    buffer('SFC' + ' ' + str(ship), 1)
    try:
        destination_loc = browser.find_element_by_xpath("//input[@type='search']")
        delayedTyping(60, LOC, destination_loc, extrakey=2)
    except NoSuchElementException as exc:
        print('Ship is currently in flight')
        return
    try:
        ship_fly_confirm()
    except NoSuchElementException as exc:
        print('Can not fly to this location')
        return
        
        
def price_data(max_price, mat, mrkt):
    print(max_price, mat, mrkt)
    #MAT.csv to reference max qty
    # TODO: Fix this
    # Not pushing to budget_df, most likely issue with comparison opperation
    # https://stackoverflow.com/questions/36921951/truth-value-of-a-series-is-ambiguous-use-a-empty-a-bool-a-item-a-any-o
    print('price_data --> get_prices')
    pricedfs = get_prices(mat, mrkt)
    df = pricedfs[0]
    dflist = df_pruner(df, 2)
    orderdf = dflist[0]
    requestdf = dflist[1]
    print(dflist[0])
    budget_df = pd.DataFrame()
    for n in range(orderdf.shape[0]):
        single_order = orderdf.iloc[-1*n-1]
        if n == orderdf.shape[0]:
            return budget_df
        else:
            if max_price < locale.atof(single_order.Price) and (budget_df.empty == True):
                print('No trade found in price bracket')
                return False
                #break
            elif max_price >= locale.atof(single_order.Price):
                print('price data added...')
                budget_df = pd.concat([budget_df, single_order], axis=1)
            else:
                print('budget data found!')
                return budget_df
    return budget_df
    
def get_prices(MAT, LOC):
    print('buffer get prices')
    buffer('CXOB' + ' ' + str(MAT) + '.' + str(LOC), 1)
    print('get table')
    dfs = get_table()
    print('table got')
    return dfs
    
def get_table():
    price_table = browser.find_elements_by_xpath("//table")
    dfs = pd.read_html(price_table[0].get_attribute('outerHTML'))
    return dfs
    
def df_pruner(df, type=None):
    # [Offers, Requests]
    def df_split(df):
        splitint = df.loc[df["Trader"]=="Requests"].index.values.astype(int)[0]
        df_offers = df.iloc[:splitint,:]
        df_requests = df.iloc[splitint:,:]
        df_requests = df_requests[df_requests["Amount"].str.contains("Requests")==False]
        df_offers = df_offers[df_offers["Amount"].str.contains("Offers")==False]
        df_requests = df_format(df_requests)
        df_offers = df_format(df_offers)
        return [df_offers, df_requests]
    def df_format(df):
        df['Trader'] = df['Trader'].astype('str')
        df['Amount'] = df['Amount'].astype('int')
        df['Price'] = df['Price'].astype('str')
        df['Currency'] = df['Currency'].astype('str')
        return df
    #shipdfs
    if type == 1:
        df.drop('Command', axis=1, inplace=True)
        df.drop('Fuel', axis=1, inplace=True)
        df.reset_index(drop=True, inplace=True)
    #pricedfs
    elif type == 2:
        df = df[df["Amount"].str.contains("Spread")==False]
        df = df[df["Amount"].str.contains("∞")==False]
        
        df[['Price','Currency']] = df.Price.str.split(expand=True)
        
        
        df.reset_index(drop=True, inplace=True)
        df = df_split(df)
    elif type == 3:
        df = df.transpose()
        df = df.sort_index(ascending=False)
        df = df.reset_index(drop=True)
    else:
        print('Error: No df type selected')
        return
    return df


def make_purchase(mat=None, max_price=None, qty=None, mrkt=None):
    
    def purchase_execute(max_price, qty, mat, mrkt, available):
        print('Executing purchase order...')
        buffer('CXPO ' + mat + '.' + mrkt, 1)
        
        if available == 1:
            print('av1')
            final_qty = str(qty)
            
        elif available == 2:
            print('av2')
            final_qty = (budget_df.Amount.sum() - qty)
            
        else:
            print("How many are available?")
            return False
        try:
            print('trying selecters')
            sleep(5)
            select = browser.find_element_by_xpath("//input[1]")
            select.send_keys(str(final_qty) + Keys.TAB + str(max_price))
            sleep(3)
            buy_btn = browser.find_element(By.XPATH, "//button[1][contains(text(), 'buy')]")
            buy_btn.click()
            buy_btn2 = browser.find_element(By.XPATH, "//button[2][contains(text(), 'place order')]")
            buy_btn2.click()
            return
        except:
            print("Purchase error[2]")
            
            
            
    try:

        mat = str(input("Enter material: "))
        max_price = float(input("Enter max price per material: "))
        qty = int(input("Enter qty: "))
        mrkt = str(input("Enter mrkt: "))
        
        print('function run --> price_data')
        budget_df = price_data(max_price, mat, mrkt)
        budget_df = df_pruner(budget_df, 3)
        display(budget_df)
        if (budget_df.Amount.sum() - qty) >= 0:
            print(f"Order can be filled fully for: {qty}  +{budget_df.Amount.sum() - qty}")
            confirm = input('Do you want to continue? [Y/n]')
            if confirm == 'Y' or confirm == 'y':
                print('Placing order...')
                purchase_execute(max_price, qty, mat, mrkt, available=1)
                print('Order placed!')
                close_buffer()
            else:
                return
        elif (budget_df.Amount.sum() - qty) < 0:
            print(f"Order can be filled partially for: {budget_df.Amount.sum()}")
            confirm = input('Do you want to continue? [Y/n]')
            if confirm == 'Y' or confirm == 'y' or confirm == '':
                print('Placing order...')
                purchase_execute(max_price, qty, mat, mrkt, available=2)
                print('Order placed!')
                close_buffer()
            else:
                return
        else:
            print(f"Order can not be filled at {mrkt}")
            return
    except:
        print("Purchase error[1]")
        return False

    
def mat_transfer():
    #buffer('MTRA', 3)
    select = Select(browser.find_element_by_xpath("//select[@class='lMW+62kRpmmuHlutv0fqBg==']"))
    select.click()
    

#ship_control('AVI-038EU', 'vh-331')
#pricedfs = get_prices('RAT', 'IC1')
#dflist = df_pruner(pricedfs, 2)
#Getting ship /  price data in df
#pricesfs = get_prices('RAT', 'IC1')
#shipdfs = ship_status()
#display(shipdfs[0])

#Functions
#login()
#ship_status()
#ship_fly(ship, LOC)
#make_purchase()
#get_prices(MAT, LOC)

#Next Functions
#Move inventory
#Sell
#Queue workers
#Global status(worker status/ finances / etc..)

In [ ]:
mat_transfer()

In [ ]:
ship_status()

In [ ]:
ship_control('AVI-038EU', 'vh-331')

In [186]:
def manage_inv():
    def create_inv_obj():
        buffer('INV', 3)
        dfs = get_table()
        df = dfs[0]
        display(df)
    def check_inv(inv_num=1):
        openbutton = browser.find_element_by_xpath(f"(//button[@class='kgGsDNvDoWj61w4I7VAlfA== fMW62cERnlzxZPFhnlPOeQ=='])[{inv_num}]")
        openbutton.click()
        inv_div = browser.find_element_by_xpath("(//div[@class='gecI5uGBluvjP5GCRk3dHA=='])[2]")
        print(inv_div.text)
        close_buffer(buffer_tab=2)
    #create_inv_obj()
    sleep(2)
    check_inv(2)

In [ ]:
manage_inv()

In [233]:
def check_inv(inv_num=1):
    openbutton = browser.find_element_by_xpath(f"(//button[@class='kgGsDNvDoWj61w4I7VAlfA== fMW62cERnlzxZPFhnlPOeQ=='])[{inv_num}]")
    openbutton.click()
    inv_div = browser.find_element_by_xpath("(//div[@class='gecI5uGBluvjP5GCRk3dHA=='])[2]")
    print(inv_div.text)
    print(type(inv_div.text))
    mystring = inv_div.text
    close_buffer(buffer_tab=2)
    return mystring

In [ ]:
mystring = check_inv(1)

In [237]:
mystring2 = mystring.splitlines()

In [ ]:
def list_slice(S, step):
    return [S[i::step] for i in range(step)]

print(list_slice(mystring2,9))
